<a href="https://colab.research.google.com/github/amovar18/machinelearningproject/blob/master/object__recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils
import sklearn 
from keras.datasets import cifar10
from PIL import Image

In [0]:
#load data as testing and training unit
(xtrain,ytrain),(xtest,ytest)=cifar10.load_data()

In [0]:
print(xtrain.shape)
print(xtest.shape)

In [0]:
#create grid of 3*3 images
for i in range(0,9):
  plt.subplot(330+1+i)
  img=xtrain[i].transpose([0,1,2])
  plt.imshow(img)
plt.show()

In [0]:
#preprocessing the dataset
#random seed for reproducablity
seed=6
np.random.seed(6)
#load data again with seed
(xtrain,ytrain),(xtest,ytest)=cifar10.load_data()
#normalize input from 0 to 255 to 0.0 to 1.0
xtrain=xtrain.astype('float32')
xtest=xtest.astype('float32')
xtrain=xtrain/255.0
xtest=xtest/255.0

In [0]:
#one hot vector for labels
#hot encode output
ytrain=np_utils.to_categorical(ytrain)
ytest=np_utils.to_categorical(ytest)
print(ytrain.shape)
print(ytrain[0])

In [0]:
#import necessary layers
from keras.models import Sequential
from keras.layers import Dropout,Activation,Conv2D,GlobalAveragePooling2D
from keras.optimizers import SGD  

In [0]:
#define model function
def all_cnn(weight=None):
  #initialize model
  model=Sequential()
  #add model layers
  model.add(Conv2D(96,(3,3),padding='same',input_shape=(32,32,3)))
  model.add(Activation('relu'))
  model.add(Conv2D(96,(3,3),padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(96,(3,3),padding='same',strides=(2,2)))
  model.add(Dropout(0.5))
  model.add(Conv2D(192,(3,3),padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(192,(3,3),padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(192,(3,3),padding='same',strides=(2,2)))
  model.add(Dropout(0.5))
  
  
  model.add(Conv2D(192,(3,3),padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(192,(1,1),padding='valid'))
  model.add(Activation('relu'))
  model.add(Conv2D(10,(1,1),padding='valid'))

  model.add(GlobalAveragePooling2D())
  model.add(Activation('softmax'))

  if weight:
    model.load_weights(weight)
  return model

In [0]:
#define hyper parameter
learnin_rate=0.01
weight_decay=1e-6
momentum=0.9
#build model
model=all_cnn()
sgd=SGD(learning_rate=learnin_rate,decay=weight_decay,momentum=momentum,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
print (model.summary())

In [0]:
epoch=350
batch_size=32
model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=epoch,batch_size=batch_size,verbose=1)

In [0]:
#define hyper parameter
learnin_rate=0.01
weight_decay=1e-6
momentum=0.9
weights="/content/all_cnn_weights_0.9088_0.4994.hdf5"
#build model
model=all_cnn(weights)
sgd=SGD(learning_rate=learnin_rate,decay=weight_decay,momentum=momentum,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
print (model.summary())
scores=model.evaluate(xtest,ytest,verbose=1)
print(scores)

In [0]:
classes=range(0,10)
names=[
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck']
class_labels=dict(zip(classes,names))
print(class_labels)

In [0]:
#generate batch of 9 images
batch=xtest[1000:1009]
labels=np.argmax(ytest[1000:1009],axis=-1)
predictions=model.predict(batch,verbose=1)

In [0]:
#convert the prediction to class probablities to class labels
class_results=np.argmax(predictions,axis=-1)
print("{}".format(class_results))

In [0]:
#create grid of 3*3 images
fig,axs=plt.subplots(3,3,figsize=(15,6))
fig.subplots_adjust(hspace=1)
axs=axs.flatten()
for i,img in enumerate(batch):
  for key,value in class_labels.items():
    if class_results[i]==key:
      title="prediction:{0} \nactual:{1}".format(class_labels[key],class_labels[labels[i]])
      axs[i].set_title(title)
      axs[i].axes.get_xaxis().set_visible(False)
      axs[i].axes.get_yaxis().set_visible(False)
  axs[i].imshow(img)
plt.show()
    